In [1]:
import numpy as np
import random
import pdb
import tensorflow as tf
from tqdm import tqdm

In [39]:
def file_read(data_dir):
    f = open(data_dir, 'r')
    lines = f.readlines()
    feature = tuple(lines[0].split('\t')[1:])
    samples = []
    data = np.zeros([100,len(feature)])
    for i, line in enumerate(lines):
        if i != 0:
            samples.append(line.split('\t')[0])
            data[i-1,:] = line.split('\t')[1:]
    f.close()
    return data, feature, samples

In [43]:
rna_data, rna_feature, samples = file_read('../data/ADD_cor_and_rna_0.35.site.ham21.ham17.ssi')
beta_data, beta_feature, _ = file_read('../data/ADD_cor_and_beta_0.35.site.ham21.ham17.ssi')

In [53]:
data = np.column_stack((rna_data,beta_data))
feature = rna_feature + beta_feature

In [49]:
label = np.zeros(100)
for i,sample in enumerate(samples):
    if 'sdpc' in sample:
        label[i] = 0
    else:
        label[i] = 1
print("size of label:", len(label))
print("# of MDD:", len(label) - len(np.nonzero(label)[0]))
print("# of normal sample:",len(np.nonzero(label)[0]))

('size of label:', 100)
('# of MDD:', 44)
('# of normal sample:', 56)


with open('../data/rna_feature.txt', 'w') as g:
    for x in feature:
        g.write(x)
        g.write('\n')

In [4]:
idx = range(len(data))
np.random.shuffle(idx)
data_x = np.array(data[idx])
label_y = np.array(label[idx])

In [26]:
data_x[:10] + data_x[20:]

ValueError: operands could not be broadcast together with shapes (10,30852) (80,30852) 

# Model

In [16]:
N = 80
itr = 1000
lr = 0.9 #learning rate
train_log = 'log_0.15/train/'
dev_log = 'log_0.15/dev/'

In [17]:
g_step = tf.Variable(0)
lr = tf.train.exponential_decay(lr, g_step, 100, 0.98)
x = tf.placeholder(tf.float32, [None, len(feature)],name="x")
y = tf.placeholder(tf.int32, [None,1],name="y")

In [18]:
W = tf.Variable(tf.zeros([len(feature),1]), name='W')
b = tf.Variable(tf.zeros(1), name="b")
l2_regularization = tf.nn.l2_loss(W) + tf.nn.l2_loss(b)
y_ = tf.matmul(x, W) + b

In [19]:
prediction = tf.greater(tf.sigmoid(y_), 0.5, name="prediction")
correct = tf.equal(prediction, tf.equal(y, True))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
loss_l2 = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.cast(y, tf.float32),logits=y_) + l2_regularization
loss_mean = tf.reduce_mean(loss_l2)
print loss_l2
train_step = tf.train.RMSPropOptimizer(lr, 0.9).minimize(loss_l2)

Tensor("add_5:0", shape=(?, 1), dtype=float32)


In [20]:
#Summary
tf.summary.scalar('loss', loss_mean)
tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge_all()

saver = tf.train.Saver()
init = tf.global_variables_initializer()
sess = tf.Session()
train_writer = tf.summary.FileWriter(train_log, sess.graph)
dev_writer = tf.summary.FileWriter(dev_log)
sess.run(init)

In [21]:
def cross_validation(x,y,portion):
    t1 = 0.2*portion
    t2 = 0.2*(portion+1)
    train_x = x[:int(len(x)*t1)] + x[int(len(x)*(t2)):]
    train_y = y[:int(len(x)*t1)] + y[int(len(x)*(t2)):]
    test_x = x[int(len(x)*t1):int(len(x)*(t2))]
    test_y = y[int(len(x)*t1):int(len(x)*(t2))]
    return train_x, train_y, test_x, test_y

In [22]:
train_x, test_x = data_x[:int(len(data_x)*0.8)], data_x[int(len(data_x)*0.8):]
train_y, test_y = label_y[:int(len(label_y)*0.8)], label_y[int(len(label_y)*0.8):]

In [23]:
def random_batch(data,label, batch_size):
    idx = np.array(range(len(data)))
    np.random.shuffle(idx)
    x = np.array(data[idx])
    y = np.array(label[idx])
    return x[:batch_size], y[:batch_size]

In [24]:
for i in range(itr):
    batch_x, batch_y = random_batch(train_x, train_y, N)
    _, acc, loss, train_summary = sess.run([train_step, accuracy, loss_mean, merged],
                                    feed_dict={x:batch_x, y:np.reshape(batch_y, (N,1))})
    if i%10 == 0:
        train_writer.add_summary(train_summary, i)
        print "batch_time: " , i , "[*] Accuracy: ", acc, ", loss: ", loss, ", lr: ", lr, "log_dir: ", train_log

InvalidArgumentError: You must feed a value for placeholder tensor 'y' with dtype int32 and shape [?,1]
	 [[Node: y = Placeholder[dtype=DT_INT32, shape=[?,1], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'y', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python2.7/dist-packages/torna

In [ ]:
test_acc, test_loss, test_summary = sess.run([accuracy, loss_mean, merged], feed_dict={x:test_x, y:np.reshape(test_y, (len(test_y),1))})
print "[*] Test Accuracy: ", test_acc , ", loss: ", test_loss, ", lr: ", lr